#1-Libraries and Datasets

In [4]:
!pip install -q dagshub
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install OpenAI
! pip install GoogleSearch
! pip install serpapi
! pip install google-search-results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s

In [6]:
import json
import pickle
import re
from collections import Counter
from typing import Any, List, Optional, Type, Union

import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from google.colab import drive
from serpapi import GoogleSearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import (StratifiedKFold, cross_val_score,
                                     train_test_split)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MultiLabelBinarizer

from langchain import LLMChain, OpenAI, PromptTemplate, SerpAPIWrapper
from langchain.agents import (AgentExecutor, AgentType, LLMSingleActionAgent,
                              Tool, initialize_agent, load_tools)
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun,
                                          CallbackManagerForToolRun)
from langchain.chains import LLMChain, SequentialChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import BaseTool
from langchain.utilities import (GoogleSearchAPIWrapper, GoogleSerperAPIWrapper,
                                  WikipediaAPIWrapper)

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier

import numpy as np

In [7]:
from google.colab import userdata
open_ai_key=userdata.get('OPENAI_API_KEY')
serper_ai_key=userdata.get('SERPER_API_KEY')

In [8]:
os.environ["OPENAI_API_KEY"] = open_ai_key
os.environ["SERPER_API_KEY"] = serper_ai_key
llm_35=OpenAI(temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [9]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [21]:
file_path_train = '/content/drive/My Drive/Omdena/Fake_News_Spanish/development_fake_news_corpus_v1_0.xlsx'

df_train = pd.read_excel(file_path_train)


# 2-Machine Learning Baseline

#2.1 Bootstrapping Simulation

In [30]:
def preprocess_text(text, remove_digits=True):
    text = re.sub(r'\W', ' ', str(text))
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    if remove_digits:
        text = re.sub(r'\d', ' ', text)
    return text.strip()


In [34]:
from sklearn.base import BaseEstimator, TransformerMixin

# Custom Transformer that applies the text preprocessing
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, remove_digits=True):
        self.remove_digits = remove_digits

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
      if isinstance(X, pd.Series):
          return X.apply(lambda x: preprocess_text(x, self.remove_digits))
      else:  # Handle other types like list
          return [preprocess_text(x, self.remove_digits) for x in X]


# Assuming `df_train` is your DataFrame containing 'Headline', 'Text', and 'Category' columns
# We first combine 'Headline' and 'Text' into a 'total_text' column for simplicity
df_train['total_text'] = df_train['Headline'] + ' ' + df_train['Text']

# Setting up the pipeline
ml_pipeline = make_pipeline(
    TextPreprocessor(remove_digits=True),  # Set remove_digits=False if digits are important
    TfidfVectorizer(max_features=5000),
    RandomForestClassifier(n_estimators=100)
)

# Fit the pipeline on the entire dataset using the combined 'total_text' column
ml_pipeline.fit(df_train['total_text'], df_train['Category'])


Pipeline(steps=[('textpreprocessor', TextPreprocessor()),
                ('tfidfvectorizer', TfidfVectorizer(max_features=5000)),
                ('randomforestclassifier', RandomForestClassifier())])

In [39]:
# Example text that you want to classify
headline_example = "Aliens are coming"
text_example = "Scientists have made a groundbreaking discovery in the field of space exploration, unveiling a new planet in a habitable zone."

# Combine the headline and text into a single string, as done during training
total_text_example = headline_example + " " + text_example

# Use the pipeline to predict the category of this new example
predicted_category = ml_pipeline.predict([total_text_example])
predicted_probability = ml_pipeline.predict_proba([total_text_example])

print("Predicted Category:", predicted_category[0])
print("Predicted Probability:", predicted_probability[0][0])


Predicted Category: Fake
Predicted Probability: 0.57


#3-Integrated System


In [40]:
# Info Extraction function
def info_extraction(subject, topic, length=100, api_key=serper_ai_key, min_search=20):
    # Define the mapping of topics to their prioritized sources
    topic_priority_map = {
        "Politics": {
            "POLITICAL PARTIES": {"name": "ACCIÓN CIUDADANA", "link": "https://accion-ciudadana.org/"},
            "TRANSPARENCY": {"name": "TRACODA", "link": "https://tracoda.info/"},
            "ELECTIONS": {"name": "VOTANTE", "link": "https://twitter.com/somosvotante"},
            "CORRUPTION": {"name": "ALAC", "link": "https://twitter.com/ALAC_SV"},
        },
        "Social": {
            "GENDER": {"name": "ORMUSA", "link": "https://ormusa.org/"},
            "VIOLENCE": {"name": "ASDEUH", "link": "https://asdehu.com/"},
            "ENVIRONMENT": {"name": "ACUA", "link": "https://www.acua.org.sv/"},
            "MIGRATION": {"name": "GMIES", "link": "https://gmies.org/"},
        },
        "Economy": {
            "BUDGET": {"name": "FUNDE", "link": "https://funde.org/"},
            "MACROECONOMY": {"name": "ICEFI", "link": "https://mail.icefi.org/etiquetas/el-salvador"},
        }
    }

    # Fetch summary from the internet using the info_extraction function
    params = {
        "engine": "google",
        "q": subject ,
        "api_key": api_key
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results["organic_results"]

    # Flatten all priority sources for secondary ranking
    all_priority_sources = {info['link'] for _, approaches in topic_priority_map.items() for _, info in approaches.items()}
    topic_linked_sources = [info['link'] for approach, info in topic_priority_map.get(topic, {}).items()]

    # Initialize summary list with rank
    summary = []
    for result in organic_results[:min_search]:
        snippet = result['snippet']
        source_url = result['link']
        words = snippet.split()[:length]  # Select the first 'length' words
        truncated_snippet = ' '.join(words)

        # Rank determination
        if source_url in topic_linked_sources:
            rank = 1  # Highest priority: Directly linked to the topic
        elif source_url in all_priority_sources:
            rank = 2  # Secondary priority: In the priority map but not directly linked to the topic
        else:
            rank = 3  # Lowest priority: Not in the priority map

        # Append the summary with metadata including rank
        summary.append({
            "snippet": truncated_snippet,
            "source": source_url,
            "rank": rank
        })

    # Sort the summary list by 'rank'
    sorted_summary = sorted(summary, key=lambda x: x['rank'])

    return sorted_summary


In [41]:
# Build Agent Wrapper
class FakeDetectionWrapper:
    def __init__(self, client):
        self.client = client

class ClassAgent(FakeDetectionWrapper):
    def __init__(self, client):
        super().__init__(client)

        self.class_agent = """ You are an agent with the task of analysing a headline {headline} .
        you will identify the subject, the event and the field the news belongs to either Politics, Economics and Social.
        you will provide a Json Structure:
          (
          "subject":  subject of the news,
          "event": event described,
         "topic": field the news belongs to Politics, Economics or Social
        )
        """
        self.prompt_class_agent = PromptTemplate(template=self.class_agent, input_variables=["headline"])

        # Corrected the LLMChain instantiation
        self.llm_chain_class_agent = LLMChain(prompt=self.prompt_class_agent, llm=self.client)

    def _run_class_branch(self,client,headline):
        self.headline=headline

        try:
            class_agent_output = self.llm_chain_class_agent.run({'headline': self.headline})
            return class_agent_output
        except Exception as e:
            print(e)
            return "Error class layer"


class FilterAgent(FakeDetectionWrapper):
    def __init__(self, client):
        super().__init__(client)

        self.filter_agent = """ You are an agent with the task of counting in how many entries of a context {context},
         an text extract{headline} can be found indentically word by word, you will review each entry
         and see if the extract {headline} can be found exactly the same within each entry,
         your job is to generate a Json structure:
              (
                "times": number of entries where the headline is found exactly word by word,
              )

        """
        self.prompt_filter_agent = PromptTemplate(template=self.filter_agent, input_variables=["headline","context"])

        # Corrected the LLMChain instantiation
        self.llm_chain_filter_agent = LLMChain(prompt=self.prompt_filter_agent, llm=self.client)

    def _run_filter_branch(self,client,headline,context):
        self.context=context
        self.headline=headline

        try:
            filter_agent_output = self.llm_chain_filter_agent.run({'headline': self.headline,'context':self.context})
            return filter_agent_output
        except Exception as e:
            print(e)
            return "Error filter layer"

class HeadlineAgent(FakeDetectionWrapper):
    def __init__(self, client):
        super().__init__(client)

        self.headline_agent = """ You are an agent with the task of identifying the whether the headline {headline} is aligned with
        the body of the news {news} you will generate a Json output
      (
        "label": 0 if they are not aligned and 1 if it is,
        "analysis": analysis the divergence between the headline and the body of the news.
      )

        """
        self.prompt_headline_agent = PromptTemplate(template=self.headline_agent, input_variables=["headline","context"])

        # Corrected the LLMChain instantiation
        self.llm_chain_headline_agent = LLMChain(prompt=self.prompt_headline_agent, llm=self.client)

    def _run_headline_branch(self,client,headline,news):
        self.headline=headline
        self.news=news

        try:
            headline_agent_output = self.llm_chain_headline_agent.run({'headline': self.headline,'news':self.news})
            return headline_agent_output
        except Exception as e:
            print(e)
            return "Error headline layer"


class DecisionAgent(FakeDetectionWrapper):
    def __init__(self, client):
        super().__init__(client)

        self.decision_agent = """You will be presented with a piece of news {news} and information gathered from the internet {filtered_context}.
        Your task is to evaluate whether the news is real or fake, based solely on:

        - Reliability of sources in context {filtered_context}.
        - Probability of the news {news} being fake as provided by probability {probability} where probability is the probability of it being fake.
        - Aligment of the headline and the news {aligment} where 0 is there is not aligment and 1 if there is.
        - Number of times the exact headline is found in other media outlets {times} with could indicate a misinformation campaign

        Based on these criteria, you must label the headline as Real or Fake, and add reasoning including the sources to support this.
        You answer strictly as single JSON string. Don't include any other verbose texts and don't include the markdown syntax anywhere.JSON format:
          (
        "category": "Fake or Real",
        "reasoning": "Your reasoning here."
           )

        """
        self.prompt_decision_agent = PromptTemplate(template=self.decision_agent, input_variables=["news","filtered_context","probability","aligment","times"])

        # Corrected the LLMChain instantiation
        self.llm_chain_decision_agent = LLMChain(prompt=self.prompt_decision_agent, llm=self.client)

    def _run_decision_branch(self,news,filtered_context,probability,aligment, times):
        self.news=news
        self.filtered_context=filtered_context
        self.probability=probability
        self.aligment= aligment
        self.times=times


        try:
            decision_agent_output = self.llm_chain_decision_agent.run({'news':self.news,'filtered_context':self.filtered_context,'probability':self.probability,'aligment':self.aligment,'times':self.times})
            return decision_agent_output
        except Exception as e:
            print(e)
            return "Error decision layer"



# 3.1 Examples

In [42]:
class ProcessPipeline:
    def __init__(self, client,input,headline,ml_pipeline):

        self.class_agent = ClassAgent(client)

        self.decision_agent = DecisionAgent(client)
        self.filter_agent=FilterAgent(client)
        self.headline_agent=HeadlineAgent(client)
        self.client = client
        self.news=input
        self.headline=headline
        self.ml_model_pipeline=ml_pipeline


    def process_news(self):
        print('intermediate Outputs')

        # Proceed with classification and summary
        try:
            class_result = self.class_agent._run_class_branch(self,self.headline)
            print("Classification Result:", class_result)
            data = json.loads(class_result)
            subject = data['subject']
            event = data['event']
            topic = data['topic']
            print(f"Subject: {subject}, Event: {event}, Topic: {topic}")
        except Exception as e:
            print(f"Error during classification: {e}")
            return {"error": "Failed to classify the news."}


        try:
            print(self.headline)
            context = info_extraction(self.headline, topic)
            print('Context:', context)


        except Exception as e:
            return {"error": "Failed during extraction."}

        # Integrate ML model predictions (Placeholder for actual model integration)
        try:
            predicted_category = self.ml_model_pipeline.predict([self.headline])  # Use the news directly or another appropriate variable
            predicted_probability = self.ml_model_pipeline.predict_proba([self.headline])[0][0]
            print(f"Predicted Category: {predicted_category}, Probability: {predicted_probability}")
        except Exception as e:
            print(f"Error during prediction: {e}")
            return {"error": "Failed to predict the news category."}

        try:
            filter_result = self.filter_agent._run_filter_branch(self,self.headline, context)
            print(filter_result)

            data = json.loads(filter_result)
            times=data['times']


        except Exception as e:
            print(f"Error during context filtering: {e}")
            return {"error": "Failed to filter the context."}

        try:
            headline_result = self.headline_agent._run_headline_branch(self,self.headline, self.news)
            #print("Filter Result:", headline_result)
            data = json.loads(headline_result)
            aligment = data["label"]
            analysis = data["analysis"]

        except Exception as e:
            print(f"Error during aligment analysis: {e}")
            return {"error": "Failed to identify aligment."}



        # Make the final decision
        try:
            decision_result = self.decision_agent._run_decision_branch(headline, context, predicted_probability,aligment,times)
            final_output = json.loads(decision_result)
            print("Decision:", final_output['category'])
            print(" ")
            print("Reasoning:",final_output['reasoning'])

            #return {"decision": final_output['category'],"reasoning":final_output['reasoning']}
        except Exception as e:
            print(f"Error during decision-making: {e}")
            return {"error": "Failed to make a final decision on the news."}


In [43]:
news = """
El candidato presidencial del partido Nuestro Tiempo, Luis Parada, dejó entrever que de llegar a ganar las próximas elecciones de 2024 tendría acercamientos con los grupos terroristas de las pandillas.
Las declaraciones del aspirante surgieron durante una entrevista en la que se le cuestionó si estaría dispuesto a tener acercamientos con los cabecillas de estos grupos criminales.«Si es necesario lo haríamos (…) hay algunas cosas que no necesitamos negociar para saberlas, otras cosas no. Podría ser necesario tener pláticas, conversaciones», fueron las declaraciones de Parada.
El militar retirado agregó también que «Cualquier plática que tengamos (con los mareros) va a ser transparente y de conocimiento público». En el pasado, entre 2012 y 2014, los gobiernos del FMLN, avalaron una tregua con las pandillas en la que se negoció, supuestamente, una reducción de homicidios, y luego negociaron votos a favor de Salvador Sánchez Cerén. Lo mismo hizo Norman Quijano, cuando competía por la presidencia por ARENA, el expartido de Parada.
Ante las declaraciones del militar retirado y ahora candidato, el ministro de Defensa, René Francis Merino Monroy, cuestionó sobre cómo es posible que el candidato de Nuestro Tiempo considere negociar con asesinos.
«Estaba viendo una entrevista que le hicieron a uno de los candidatos, y le preguntan que si él negociaría con pandillas, entonces el candidato dice: ´Sí; hay cosas que hay que revisarlas con las pandillas y hay que ponernos de acuerdo´», comentó el ministro durante la entrevista de Diálogo de Canal 21.
Merino Monroy añadió que «La población ya sabe que esas negociaciones por más que digan que las van a hacer de cara a la población, ¿cómo se va a poner a negociar con asesinos?».
Los acercamientos que los gobiernos de los partidos ARENA y FMLN tuvieron con las pandillas permitió que estos grupos criminales se fortalecieran y tomaran el control total de la mayoría de las colonias y comunidades del país.
La denominada tregua entre pandillas, avalada por el primer gobierno del FMLN, concedió todo tipo de privilegios a estos grupos terroristas, mientras la población honrada y trabajadora era asesinada ante la mirada pasiva de las autoridades de turno.
Tras viralizarse las declaraciones de Parada sobre su aval para que negociar con pandilleros, luego posteó en la red «X», para matizar que «podría ser necesario hablar con los líderes pandilleros» para consultar sobre supuestas negociaciones con el gobierno, a pesar que el gobierno de Bukele ha sido el único que en 30 años declaró la guerra a las pandillas y ha encarcelado a más de 70,000 terroristas y hasta ordenó construir el Cecot, la cárcel exclusiva para mareros.
"""
headline='El candidato presidencial Luis Parada propone dialogar con las pandillas'

verification_process = ProcessPipeline(llm_35,news,headline,ml_pipeline)


# Or, to process without using the filter agent
result = verification_process.process_news()


print(result)

intermediate Outputs


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Classification Result: 
{
  "subject": "Luis Parada",
  "event": "propone dialogar con las pandillas",
  "topic": "Politics"
}
Subject: Luis Parada, Event: propone dialogar con las pandillas, Topic: Politics
El candidato presidencial Luis Parada propone dialogar con las pandillas
Context: [{'snippet': 'En el pasado, entre 2012 y 2014, los gobiernos del FMLN, avalaron una tregua con las pandillas en la que se negoció, supuestamente, una ...', 'source': 'https://diarioelsalvador.com/el-candidato-presidencial-luis-parada-propone-dialogar-con-las-pandillas/400083/', 'rank': 3}, {'snippet': 'Nación TV | El candidato presidencial Luis Parada propone dialogar con las pandillas. 366 views · 5 months ago ...more ...', 'source': 'https://www.youtube.com/watch?v=-QMkmGtF8rc', 'rank': 3}, {'snippet': 'Lo que llevó la paz a El Salvador es un pacto corrupto con las pandillas. Luis Parada. Decidimos que era necesario participar porque hay que ...', 'source': 'https://legrandcontinent.eu/es/2024/02/04

In [ ]:
news = """
El también presidente de la Comisión de Hacienda de la Asamblea y jefe de la banca de Nuevas Ideas dijo,
sin embargo, que no fue una elección de “foto finish”, su partido venció con claridad y mucha distancia y los perdedores
siempre van a hablar, en relación a los reclamos para sembrar las dudas en el recién concluido proceso.
Con la inversión que se hizo y los fondos aprobados al TSE “esperábamos una elecciones inmaculadas”,
señaló Guevara en sus críticas al organismo que pasó a ser la “oveja negra” en el proceso electoral.
No hay dudas en la victoria de Nuevas Ideas y por eso apoyamos la decisión del TSE y los partidos políticos
del conteo voto a voto que comenzará hoy, sostuvo durante una comparecencia en el programa de entrevista
Frete a Frente de la Telecorporación Salvadoreña (TCS). No hay dudas en la victoria de Nuevas Ideas
y por eso apoyamos la decisión del TSE y los partidos políticos del conteo voto a voto que comenzará hoy,
sostuvo durante una comparecencia en el programa de entrevista Frete a Frente de la Telecorporación Salvadoreña (TCS).
Guevara pidió a la oposición que aporte pruebas sobre las violaciones del proceso electoral que
son objetos de denuncia por Arena y otras agrupaciones que participaron en los comicios.
Dijo que ve muy poco probable que cambie el pronóstico de votos para el Legislativo, no obstante,
subrayó, vamos por las 60 butacas. Asimismo manifestó su apoyo a repetir las elecciones en algún lugar
específico donde se comprueben fallas pero no a nivel nacional donde es “evidente” la ventaja de Nuevas ideas.
En estas acciones que denuncia la oposición, aseguró que el más afectado es el partido Nuevas Ideas, pues las votaciones
muestran su aplastante victoria.Por otra parte, el TSE informó que suspendieron la repetición de la votación el
exterior que habían anunciado la víspera, la decisión se tomó luego de una reunión con los partidos políticos y tras las
observaciones y cuestionamientos del partido Arena en relación con que no se lleve a cabo la elección extraordinaria.
Este miércoles comenzará el conteo voto a voto de las boletas, en especial las que decidirán la integración de la
Asamblea Legislativa. La institución indicó que se llevará a cabo la instalación de 49 mesas encargadas del voto nacional, mientras que dos más serán instaladas para el voto en el exterior en un proceso que demorará entre 10 y 15 días, según fuentes del tribunal. La institución indicó que se llevará a cabo la instalación de 49 mesas encargadas del voto nacional, mientras que dos más serán instaladas para el voto en el exterior en un proceso que demorará entre 10 y 15 días, según fuentes del tribunal. La presidenta del ente rector de los comicios, Dora Martínez,
aseguró que el proceso será fiscalizado, transparente y de conocimiento público para los electores.

"""

headline='Afirman que TSE en El Salvador tuvo actuación deplorable'
verification_process = ProcessPipeline(llm_35, news,headline,ml_pipeline)


# Or, to process without using the filter agent
result = verification_process.process_news()

print(result)

intermediate Outputs
Classification Result: 
{
  "subject": "TSE en El Salvador",
  "event": "actuación deplorable",
  "topic": "Politics"
}
Subject: TSE en El Salvador, Event: actuación deplorable, Topic: Politics
Afirman que TSE en El Salvador tuvo actuación deplorable
Context: [{'snippet': 'San Salvador, 7 feb (Prensa Latina) El Tribunal Supremo Electoral (TSE) de El Salvador tuvo una actuación deplorable durante las elecciones ...', 'source': 'https://www.prensa-latina.cu/2024/02/07/afirman-que-tse-en-el-salvador-tuvo-actuacion-deplorable/', 'rank': 3}, {'snippet': 'Mientras tanto el TSE está por concluir ... Relacionado. Afirman que TSE en El Salvador tuvo actuación deplorable 8 febrero, 2024 En «Destacadas».', 'source': 'https://www.elindependiente.sv/2024/02/09/prosiguen-en-el-salvador-criticas-a-proceso-de-elecciones/', 'rank': 3}, {'snippet': 'San Salvador, 7 feb (Prensa Latina) El Tribunal Supremo Electoral (TSE) de El Salvador tuvo una actuación deplorable durante las elecci

In [ ]:
news = """
Los magistrados dijeron en su resolución que con la prueba pericial y documental los fiscales de la Unidad Anticorrupción les demostraron el enriquecimiento ilícito de forma indebida entre el 2012 al 2018.


Uno de los magistrados sostuvo que valoraron toda la prueba, entre ellos dos informes de pericia financiera contable que acreditó el aumento patrimonial sin justificación de las dos demandadas.

Los fiscales demandaron por 15 irregularidades, pero al final solo se probaron ocho.

A Lorena Peña la condenaron por un incremento patrimonial de $133,746.60 , en cinco irregularidades, mientras que su hija le hallaron tres inconsistencias por la cantidad de $17,016.26.

Durante la audiencia probatoria, los fiscales le manifestaron a los magistrados que en el análisis que hizo la Sección de Probidad de la Corte Suprema de Justicia (CSJ) encontró las ocho inconsistencias de movimientos financieros, compra de inmuebles y pagos mediante depósitos bancarios de tarjetas de crédito.

Todos esos pagos y adquisiciones son desproporcionados ya que no están de acuerdo a los ingresos legítimos percibidos por la exdiputada y su hija. En la decisión judicial se le ha dado la razón a la Fiscalía accediendo a una condena.

Ambas están obligadas a devolver al Estado el monto con el cual se enriquecieron indebidamente, además, quedan inhabilitadas para ejercer un cargo público durante un plazo de 10 años, como lo señala la Ley de Enriquecimiento Ilícito de 1959.


El juicio civil fue ordenado por nueve de 15 magistrados de la Corte Suprema de Justicia, el 26 de enero de 2023, luego de un informe que envió Probidad donde se señaló que la exlegisladora y su hija no pudieron justificar los ingresos cuestionados en este proceso.

Lorena Peña, enfrentó el proceso por el incremento patrimonial que tuvo como diputada propietaria de la Asamblea Legislativa, en los periodos del 1 de mayo del año 2012 al 30 de abril del 2015 y del 1 de mayo de 2015 al 30 de abril del 2018.

La exdiputada se convierte en la sexta exfuncionaria de los gobiernos del FMLN en ser condenada por enriquecimiento ilícito y en el 2024 es la segunda, ya que en enero fue condenado Medardo González, exsecretario general de ese instituto político.

En el 2023 fueron encontrados responsables de enriquecimiento ilícito José Atilio Benítez, exministro de Defensa; Benito Lara, exdiputado y exministro de Justicia y Seguridad; David Munguía Payés, exministro de la Defensa y Eduardo Linares, exdirector del Organismo de Inteligencia del Estado (OIE).
"""

headline='Exdiputada Lorena Peña y su hija son condenadas por enriquecimiento ilícito por más de $150,000'

verification_process = ProcessPipeline(llm_35, news,headline,ml_pipeline)


# Or, to process without using the filter agent
result = verification_process.process_news()

print(result)

intermediate Outputs
Classification Result: 
{
  "subject": "Exdiputada Lorena Peña y su hija",
  "event": "son condenadas por enriquecimiento ilícito por más de $150,000",
  "topic": "Politics"
}
Subject: Exdiputada Lorena Peña y su hija, Event: son condenadas por enriquecimiento ilícito por más de $150,000, Topic: Politics
Exdiputada Lorena Peña y su hija son condenadas por enriquecimiento ilícito por más de $150,000
Context: [{'snippet': 'Lorena Peña e hija son condenadas por enriquecimiento ilícito de $150,000 · condenó a la exdiputada del FMLN, Lorena Peña y a su hija Ana ...', 'source': 'https://diario.elmundo.sv/nacionales/lorena-pena-e-hija-son-condenadas-por-enriquecimiento-ilicito-de-150000', 'rank': 3}, {'snippet': 'La exdiputada se convierte en la sexta exfuncionaria de los gobiernos del FMLN en ser condenada por enriquecimiento ilícito y en el 2024 es la ...', 'source': 'https://diarioelsalvador.com/exdiputada-lorena-pena-y-su-hija-son-condenadas-por-enriquecimiento-ilicit

In [ ]:
news = """
El Tribunal Supremo Electoral (TSE) entregó este jueves, la acreditación al Presidente Nayib Bukele para su segundo mandato en El Salvador, evento que contó con la participación de todo el Cuerpo Diplomático.

«Todo el Cuerpo Diplomático acreditado en El Salvador acompañó este día al Presidente Electo, Nayib Bukele y al vicepresidente, Félix Ulloa, en el acto de la Certificación de Resultados de las Elecciones Presidenciales 2024», detalló Casa Presidencial en su cuenta de «X».

El Cuerpo Diplomático está conformado por un total de 40 representantes de países de todo el mundo, quienes mostraron su respaldo a la reelección del Mandatario salvadoreño al estar presente en el evento de Certificación de Resultados de las Elecciones Presidenciales 2024.
"""

headline=" Cuerpo Diplomático acompañó la acreditación del segundo mandato del Presidente Bukele"
verification_process = ProcessPipeline(llm_35, news,headline,ml_pipeline)

# Or, to process without using the filter agent
result = verification_process.process_news()

print(result)

intermediate Outputs
Classification Result: 
{
  "subject": "Cuerpo Diplomático",
  "event": "acreditación del segundo mandato del Presidente Bukele",
  "topic": "Politics"
}
Subject: Cuerpo Diplomático, Event: acreditación del segundo mandato del Presidente Bukele, Topic: Politics
 Cuerpo Diplomático acompañó la acreditación del segundo mandato del Presidente Bukele
Context: [{'snippet': 'El Tribunal Supremo Electoral (TSE) entregó este jueves, la acreditación al Presidente Nayib Bukele para su segundo mandato en El Salvador, ...', 'source': 'https://ultimahora.sv/cuerpo-diplomatico-acompano-la-acreditacion-del-segundo-mandato-del-presidente-bukele/', 'rank': 3}, {'snippet': 'Comments3 · Recepción al Cuerpo Diplomático acreditado en España · Dispositivo de controles antidopaje deja más de una decena de detenidos en las ...', 'source': 'https://www.youtube.com/watch?v=uV8FKs1V1vU', 'rank': 3}, {'snippet': 'Este 29 de febrero, se llevó a cabo la Certificación de los Resultados Presidenc

In [ ]:
news = """
Autoridades de la Dirección de Obras Municipales (DOM) lanzaron el proyecto de mejoramiento vial en la calle principal que conecta a los cantones El Guayabo y Cutumayo, en Apastepeque, San Vicente.

«Desde hoy, las cuadrillas de la Dirección de Obras Municipales se han enfocado en la renovación de 8.4 kilómetros de calles, beneficiando a más de 20,000 residentes locales», informó la entidad.

El objetivo es mejorar la movilidad y la conexión en la zona para permitir el desarrollo económico local, con un acceso en óptimas condiciones, donde los agricultores podrán movilizar sus cosechas de forma más fácil a los puntos de venta.

"""

headline='Lanzan proyecto de mejoramiento vial en zona rural de San Vicente'
verification_process = ProcessPipeline(llm_35, news,headline,ml_pipeline)

# Or, to process without using the filter agent
result = verification_process.process_news()

print(result)

intermediate Outputs
Classification Result: 
{
  "subject": "proyecto de mejoramiento vial",
  "event": "lanzamiento",
  "topic": "Economics"
}
Subject: proyecto de mejoramiento vial, Event: lanzamiento, Topic: Economics
Lanzan proyecto de mejoramiento vial en zona rural de San Vicente
Context: [{'snippet': '#Nacionales | Lanzan proyecto de mejoramiento vial en zona rural de San Vicente Nota completa: https://ultimahora.sv/lanzan-proyecto-de ...', 'source': 'https://twitter.com/ultimahsv/status/1766540564437065919', 'rank': 3}, {'snippet': '#Nacionales Autoridades de la Dirección de Obras Municipales (DOM) lanzaron el proyecto de mejoramiento vial en la calle principal que conecta a los cantones El ...', 'source': 'https://m.facebook.com/story.php/?story_fbid=803232658511928&id=100064756023835', 'rank': 3}, {'snippet': '#Nacionales Autoridades de la Dirección de Obras Municipales (DOM) lanzaron el proyecto de mejoramiento vial en la calle principal que conecta a los cantones El ...', '

In [ ]:
news = """
El medio internacional Voz Media USA ha destacado que el Modelo Bukele está inspirando a los países de la región, siendo Argentina uno de los últimos en retomar la estrategia que el Presidente salvadoreño ejecuta en su país.

«Bukele goza de una popularidad nunca vista en la región gracias a su eficiente modelo de seguridad, que funcionó tanto para reducir los homicidios en mínimos históricos como para controlar las pandillas», destaca el medio en un artículo periodístico.

En Santa Fe, Argentina, las autoridades pusieron en marcha dicho modelo, donde tras un ataque a un bus con efectivos penitenciarios desató una sorpresiva requisa en la cárcel de la localidad, los internos fueron atados de sus brazos y tomaron el control del recinto.

Otros países que se han inspirado en el modelo Bukele son Honduras y Ecuador, que han sido atacados por grupos criminales dejando decenas de fallecidos con el accionar en la ciudad y lugares recónditos.



"""

headline='El modelo Bukele inspira a la región'
verification_process = ProcessPipeline(llm_35, news,headline,ml_pipeline)

# Or, to process without using the filter agent
result = verification_process.process_news()

print(result)

intermediate Outputs
Classification Result: 
{
  "subject": "El modelo Bukele",
  "event": "inspira a la región",
  "topic": "Politics"
}
Subject: El modelo Bukele, Event: inspira a la región, Topic: Politics
El modelo Bukele inspira a la región
{'error': 'Failed during extraction.'}


In [ ]:
llm_pred=[]
for i in range(df_eval.shape[0]):
  print(i)
  verification_process = ProcessPipeline(llm_35, df_eval.loc[i,"text"],df_eval.loc[i,"title"],ml_pipeline)
  result = verification_process.process_news()
  print('Real label', df_eval.loc[i,'label'])


0
intermediate Outputs
Classification Result: 
{
  "subject": "TSE en El Salvador",
  "event": "actuación deplorable",
  "topic": "Politics"
}
Subject: TSE en El Salvador, Event: actuación deplorable, Topic: Politics
Afirman que TSE en El Salvador tuvo actuación deplorable
Context: [{'snippet': 'San Salvador, 7 feb (Prensa Latina) El Tribunal Supremo Electoral (TSE) de El Salvador tuvo una actuación deplorable durante las elecciones ...', 'source': 'https://www.prensa-latina.cu/2024/02/07/afirman-que-tse-en-el-salvador-tuvo-actuacion-deplorable/', 'rank': 3}, {'snippet': 'Mientras tanto el TSE está por concluir ... Relacionado. Afirman que TSE en El Salvador tuvo actuación deplorable 8 febrero, 2024 En «Destacadas».', 'source': 'https://www.elindependiente.sv/2024/02/09/prosiguen-en-el-salvador-criticas-a-proceso-de-elecciones/', 'rank': 3}, {'snippet': 'San Salvador, 7 feb (Prensa Latina) El Tribunal Supremo Electoral (TSE) de El Salvador tuvo una actuación deplorable durante las elec

In [ ]:
result

{'error': 'Failed to filter the context.'}

# 4.UI Testing

In [3]:
# Refactored Code

# Imports
import json
import re
import os
import pickle
from typing import List
import numpy as np
import pandas as pd
import torch
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample

# Constants
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
SERPER_API_KEY = os.environ.get('SERPER_API_KEY')

# Mount Google Drive
def mount_google_drive():
    from google.colab import drive
    drive.mount('/content/drive')

# Load data
def load_data(file_path: str) -> pd.DataFrame:
    df = pd.read_excel(file_path)
    return df

# Preprocess text
def preprocess_text(text: str) -> str:
    text = re.sub(r'\W', ' ', str(text))
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\d', ' ', text)
    return text

# Encode features
def encode_features(X_train: List[str], X_test: List[str], max_features: int = 5000):
    tfidf = TfidfVectorizer(max_features=max_features, preprocessor=preprocess_text)
    X_train_enc = tfidf.fit_transform(X_train)
    X_test_enc = tfidf.transform(X_test)
    return X_train_enc, X_test_enc

# Train model
def train_model(X_train_enc: np.array, y_train: List[str]):
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train_enc, y_train)
    return model

# Predict categories
def predict_categories(model, headlines: List[str]) -> List[str]:
    predicted_categories = model.predict(headlines)
    return predicted_categories

# Extract information
def info_extraction(subject: str, topic: str, length: int = 100, api_key: str = SERPER_API_KEY, min_search: int = 20):
    # Your info_extraction logic here
    pass

# Streamlit UI
import streamlit as st

def main():
    st.title("Fake News Detection")

    # Sidebar
    st.sidebar.header("Settings")
    file_path_train = st.sidebar.text_input("Train Data File Path")
    file_path_eval = st.sidebar.text_input("Eval Data File Path")
    openai_api_key = st.sidebar.text_input("OpenAI API Key")
    serper_api_key = st.sidebar.text_input("Serper API Key")

    # Mount Google Drive
    mount_google_drive()

    # Load data
    df_train = load_data(file_path_train)
    df_eval = load_data(file_path_eval)

    # Preprocessing
    df_train.dropna(subset=['Category', 'Headline', 'Text'], inplace=True)
    df_eval.dropna(subset=['title', 'text', 'label'], inplace=True)
    df_train['total'] = df_train['Headline'] + ' ' + df_train['Text']
    df_train['input'] = df_train['total'].apply(preprocess_text)
    df_eval['label'] = df_eval['label'].replace({'Real': True})
    df_eval['label'] = df_eval['label'].astype(str)
    df_eval.dropna(subset=['title', 'text'], inplace=True)
    df_eval['total'] = df_eval['title'] + ' ' + df_eval['text']
    df_eval['input'] = df_eval['total'].apply(preprocess_text)

    # Encode features
    X_train_enc, X_test_enc = encode_features(df_train['input'].tolist(), df_eval['input'].tolist())

    # Train model
    model = train_model(X_train_enc, df_train['Category'])

    # Sidebar - Input
    st.sidebar.header("News Input")
    new_headlines_input = st.sidebar.text_area("Enter new headlines (one per line)")

    if st.sidebar.button("Predict"):
        # Predict categories
        new_headlines = new_headlines_input.split("\n")
        predicted_categories = predict_categories(model, new_headlines)
        for headline, category in zip(new_headlines, predicted_categories):
            st.write(f"Headline: '{headline}', Predicted Category: {category}")

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'streamlit'

In [ ]:
# Save into repository
from dagshub.notebook import save_notebook

save_notebook(repo="Omdena/IREX-El-Salvador", path="task_3-model-building")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=2f19347b-8e4b-4a82-ab18-468ea8d44d66&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=3d1f253f3caafc8b72b9914170faeb8826e4dbf5ee4abee6ae04c8af537abef2




Uploading files (1) to "Omdena/IREX-El-Salvador"...

Upload finished successfully!